# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [2]:
# Imports
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
import seaborn as sns
import sqlite3
import time

from bs4 import BeautifulSoup

In [3]:
# Get exceptions msg function
def get_exception_msg(exception, err):
    exceptions = {
        "timeout": "It has taken too long and your request has expired",
        "connectionError": "Connection error. Check your internet connection.",
        "httpError": f"HTTP error: {err}",
        "unexpected": f"An unexpected error occurred: {err}"
    }

    return exceptions[exception]

In [5]:
# Get HTML data
url = "https://ycharts.com/companies/TSLA/revenues"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

try:
    time.sleep(10)
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    web_data = response.text
except requests.exceptions.Timeout:
    print(get_exception_msg('timeout', ''))
except requests.exceptions.ConnectionError:
    print(get_exception_msg('connectionError', ''))
except requests.exceptions.HTTPError as err:
    print(get_exception_msg('httpError', err))
except Exception as err:
    print(get_exception_msg('unexpected', err))

In [8]:
# Create DataFrame with data
html_parsed_data = BeautifulSoup(web_data, "html.parser")
panel_content = html_parsed_data.findAll("div", class_="panel-content")

if panel_content:
    for each in panel_content:
        row_divs = each.findAll("div", class_="row")
        
        for each in row_divs:
            tables = each.findAll("table")

df = pd.DataFrame(columns = ["Date", "Value"])

for each in tables:
    rows = each.findAll("tr")

    for each in rows:
        cells = each.findAll("td")

        if len(cells) == 2:
            date = cells[0].get_text(strip=True)
            value = cells[1].get_text(strip=True)

            df = pd.concat([df, pd.DataFrame({"Date": [date], "Value": [value]})], ignore_index=True)

df.head()


,Date,Value
0,"September 30, 2024",25.18B
1,"June 30, 2024",25.50B
2,"March 31, 2024",21.30B
3,"December 31, 2023",25.17B
4,"September 30, 2023",23.35B


In [12]:
# Clean DataFrame
def parse_to_numeric(value):
    if pd.isna(value):
        return None
    value = str(value)

    if "B" in value:
        return float(value.replace("B", "").strip()) * 1e9
    elif "M" in value:
        return float(value.replace("M", "").strip()) * 1e6
    else:
        return float(value)

df_clean = df.dropna()
df_clean["Date"] = pd.to_datetime(df_clean["Date"], format="%B %d, %Y")
df_clean["Value"] = df_clean['Value'].apply(parse_to_numeric)
    
print(df_clean.dtypes)
print(df_clean.head())

Date     datetime64[ns]
Value           float64
dtype: object
        Date         Value
0 2024-09-30  2.518000e+10
1 2024-06-30  2.550000e+10
2 2024-03-31  2.130000e+10
3 2023-12-31  2.517000e+10
4 2023-09-30  2.335000e+10


In [14]:
# Create and connect DB
conexion = sqlite3.connect("web_scraping.db")

# Cargar el DataFrame en SQLite, creando una tabla llamada 'data'
df.to_sql("data", conexion, if_exists="replace", index=False)

cursor = conexion.cursor()
cursor.execute("SELECT * FROM data")
resultados = cursor.fetchall()

print("Df:")
for each in resultados:
    print(each)

conexion.close()

Df:
('September 30, 2024', '25.18B')
('June 30, 2024', '25.50B')
('March 31, 2024', '21.30B')
('December 31, 2023', '25.17B')
('September 30, 2023', '23.35B')
('June 30, 2023', '24.93B')
('March 31, 2023', '23.33B')
('December 31, 2022', '24.32B')
('September 30, 2022', '21.45B')
('June 30, 2022', '16.93B')
('March 31, 2022', '18.76B')
('December 31, 2021', '17.72B')
('September 30, 2021', '13.76B')
('June 30, 2021', '11.96B')
('March 31, 2021', '10.39B')
('December 31, 2020', '10.74B')
('September 30, 2020', '8.771B')
('June 30, 2020', '6.036B')
('March 31, 2020', '5.985B')
('December 31, 2019', '7.384B')
('September 30, 2019', '6.303B')
('June 30, 2019', '6.35B')
('March 31, 2019', '4.541B')
('December 31, 2018', '7.226B')
('September 30, 2018', '6.824B')
('June 30, 2018', '4.002B')
('March 31, 2018', '3.409B')
('December 31, 2017', '3.288B')
('September 30, 2017', '2.985B')
('June 30, 2017', '2.790B')
('March 31, 2017', '2.696B')
('December 31, 2016', '2.285B')
('September 30, 2016'